In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

model_name='Llama2-7b-hf'
#model_name='Llama-3.2-1B'

DTYPE = torch.float16
model = qlib.load_custom_llama(
	model_name=model_name, 
	device_map='cpu', 
	torch_dtype=DTYPE, 
	#use_flash_attn=True
)

LOL, Im custom!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f7f82d2a830>>
Traceback (most recent call last):
  File "/home/msst/miniconda3/envs/qenv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [ ]:
T=256 
L=16
V=2
K=2
tlut_bits=10
decode_mode='quantlut_sym_2d'
incoh_proc_mode='qtip'
viterby_batch_size=256#1024


wrapper = qlib.HomeQuantWrapper(
    wrap_rule={
        torch.nn.Linear : qlib.TrellisLinear(
			T=T, 
            L=L, 
            V=V, 
            K=K,
			tlut_bits=tlut_bits,
            decode_mode=decode_mode, 
            incoh_proc_mode=incoh_proc_mode, 
            viterby_batch_size=viterby_batch_size
		),
    },
    exceptions = {
        'lm_head' : None,
    }
)
wrapper.wrap_model(model)

model_checkpoints_path = f'/home/msst/repo/Quantization/logs/checkpoints_{model_name}/trellis'
os.makedirs(model_checkpoints_path, exist_ok=True)
torch.save(model, os.path.join(model_checkpoints_path, f'T{T}_L{L}_V{V}_K{K}_{decode_mode}_{incoh_proc_mode}.pth'))


100%|██████████| 688/688 [00:53<00:00, 12.87it/s]
